In [ ]:
import time
import requests
from bs4 import BeautifulSoup
import csv
import schedule
from IPython.display import display, clear_output

def crawl_and_save():
    """네이버 경제 뉴스 크롤링 후 CSV로 저장"""
    url = "https://news.naver.com/section/101"
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # HTTP 에러 확인
    except requests.exceptions.RequestException as e:
        print(f"❌ 요청 오류 발생: {e}")
        return

    soup = BeautifulSoup(response.text, "html.parser")

    # ✅ 최대 70개까지 가져오고, 실제 저장은 50개만
    news_list = soup.select("div.sa_text a")[:70]  

    def get_article_content(url):
        """기사 본문 가져오기"""
        headers = {"User-Agent": "Mozilla/5.0"}
        try:
            response = requests.get(url, headers=headers)
            response.raise_for_status()
        except requests.exceptions.RequestException:
            return "❌ 본문을 가져올 수 없습니다."
        
        soup = BeautifulSoup(response.text, "html.parser")
        possible_selectors = ["div#newsct_article", "div#articleBodyContents", "div#dic_area"]
        
        for selector in possible_selectors:
            article = soup.select_one(selector)
            if article:
                return article.get_text(strip=True)
        
        return "❌ 본문을 찾을 수 없습니다."

    news_data = []
    for news in news_list:
        title = news.get_text(strip=True)
        link = news["href"]

        # URL 정리
        if not link.startswith("http"):
            article_url = "https://news.naver.com" + link
        else:
            article_url = link

        # 필요 없는 뉴스 필터링
        if "더보기" in title or len(title) < 5:
            continue
        
        article_content = get_article_content(article_url)
        if "❌ 본문" in article_content:
            continue

        news_data.append((title, article_url, article_content))

        # ✅ 뉴스 저장 개수를 50개로 변경
        if len(news_data) >= 50:
            break

    # CSV 파일로 저장 (덮어쓰기 모드)
    csv_filename = r"C:\pthon_basic\webcrolingProject\news_crawler_project\data\naver_news.csv"
    with open(csv_filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["제목", "링크", "본문"])
        writer.writerows(news_data)

    # 화면 클리어 & 진행 상태 업데이트
    clear_output(wait=True)
    print(f"✅ {csv_filename} 파일이 업데이트되었습니다! (50개 뉴스 저장)")
    display_news(news_data)

def display_news(news_data):
    """크롤링된 뉴스 제목 10개만 미리보기"""
    print("\n📢 최신 뉴스 미리보기:")
    for idx, (title, link, _) in enumerate(news_data[:20], 1):
        print(f"{idx}. {title} ({link})")

# ✅ 5분마다 실행하도록 설정
schedule.every(5).minutes.do(crawl_and_save)

print("✅ Jupyter Notebook에서 5분마다 뉴스 크롤링이 실행됩니다.")

# **5분마다 실행되는 루프 (Jupyter 실행 중지하면 멈춤)**
while True:
    schedule.run_pending()
    time.sleep(60)  # 1분마다 실행 여부 확인


✅ C:\pthon_basic\webcrolingProject\news_crawler_project\data\naver_news.csv 파일이 업데이트되었습니다! (50개 뉴스 저장)

📢 최신 뉴스 미리보기:
1. 증시 활황·민생추경에 소비심리↑…“美관세에 하방압력 여전” (https://n.news.naver.com/mnews/article/018/0006068500)
2. 롯데웰푸드, 서울 마포서 마라톤 대회 ‘2025 설레임런’ 개최 (https://n.news.naver.com/mnews/article/015/0005159634)
3. ‘잃어버린 9년’ 무거운 짐 벗고 뉴삼성 속도낸다 (https://n.news.naver.com/mnews/article/022/0004052608)
4. 관세 불안 잠재운 '소매 판매'…S&P·나스닥 최고치 [뉴욕증시 브리핑] (https://n.news.naver.com/mnews/article/015/0005159522)
5. SGI서울보증, 보증서 발급 재개…주요 전산시스템 복구 (https://n.news.naver.com/mnews/article/629/0000408165)
6. 무신사 스탠다드, 서울 동부 첫 매장 '더리버몰 강동점' 개점 (https://n.news.naver.com/mnews/article/008/0005223540)
7. GS칼텍스, KAIST와 '다문화 과학인재양성 캠프' 개최 (https://n.news.naver.com/mnews/article/001/0015514915)
8. "차 고쳐달라고 맡겼더니 손상"‥소비자원 신청만 953건 (https://n.news.naver.com/mnews/article/214/0001437144)
9. 中-日 관광 급증에, 인천공항이 수혜..역대 최다 경신 (https://n.news.naver.com/mnews/article/016/0002501458)
10. 더스프링시니어, TV광고 캠페인 온에어 (https://n.news.naver.com

In [3]:
import time
import requests
from bs4 import BeautifulSoup
import csv

# ✅ 네이버 경제 뉴스 URL
URL = "https://news.naver.com/section/101"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_article_content(url):
    """📌 네이버 뉴스 기사 본문 가져오기"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
    except requests.exceptions.RequestException:
        return "❌ 본문을 가져올 수 없습니다."
    
    soup = BeautifulSoup(response.text, "html.parser")
    possible_selectors = ["div#newsct_article", "div#articleBodyContents", "div#dic_area"]

    for selector in possible_selectors:
        article = soup.select_one(selector)
        if article:
            return article.get_text(strip=True)
    
    return "❌ 본문을 찾을 수 없습니다."

def crawl_naver_news():
    """📌 네이버 경제 뉴스 크롤링 (100개)"""
    try:
        response = requests.get(URL, headers=HEADERS)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ 요청 오류 발생: {e}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    news_list = soup.select("div.sa_text a")[:150]  # ✅ 최대 150개 크롤링 후 필터링

    news_data = []
    for news in news_list:
        title = news.get_text(strip=True)
        link = news["href"]

        # URL 정리
        if not link.startswith("http"):
            article_url = "https://news.naver.com" + link
        else:
            article_url = link

        # 필요 없는 뉴스 필터링
        if "더보기" in title or len(title) < 5:
            continue
        
        # ✅ 기사 본문 가져오기
        article_content = get_article_content(article_url)
        if "❌ 본문" in article_content:
            continue

        news_data.append((title, article_url, article_content))

        # ✅ 뉴스 100개가 채워지면 종료
        if len(news_data) >= 100:
            break

    return news_data

def save_news_to_csv(news_data, filename="naver_news.csv"):
    """📌 크롤링한 뉴스 데이터를 CSV로 저장"""
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["제목", "링크", "본문"])
        writer.writerows(news_data)

    print(f"✅ {filename} 파일이 업데이트되었습니다! (100개 뉴스 저장)")

# ✅ 실행
start_time = time.time()
news_data = crawl_naver_news()
save_news_to_csv(news_data)
end_time = time.time()

print(f"⏳ 실행 완료! 걸린 시간: {round(end_time - start_time, 2)}초")


✅ naver_news.csv 파일이 업데이트되었습니다! (100개 뉴스 저장)
⏳ 실행 완료! 걸린 시간: 7.54초


In [4]:
import os
import time

csv_filename = "naver_news.csv"

if os.path.exists(csv_filename):
    modified_time = os.path.getmtime(csv_filename)
    print("📅 마지막 업데이트 시간:", time.ctime(modified_time))


📅 마지막 업데이트 시간: Sat Mar  8 16:44:19 2025


In [2]:
import time
import requests
from bs4 import BeautifulSoup
import csv

# ✅ 네이버 경제 뉴스 URL
URL = "https://news.naver.com/section/101"
HEADERS = {"User-Agent": "Mozilla/5.0"}

def get_article_content_and_date(url):
    """📌 네이버 뉴스 기사 본문 + 날짜 가져오기"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
    except requests.exceptions.RequestException:
        return "❌ 본문을 가져올 수 없습니다.", "❌ 날짜 없음"
    
    soup = BeautifulSoup(response.text, "html.parser")
    possible_selectors = ["div#newsct_article", "div#articleBodyContents", "div#dic_area"]
    date_selector = ["span#newsct_article_date", "span.media_end_head_info_datestamp_time", "span.article_date"]

    # ✅ 기사 본문 가져오기
    article_content = "❌ 본문을 찾을 수 없습니다."
    for selector in possible_selectors:
        article = soup.select_one(selector)
        if article:
            article_content = article.get_text(strip=True)
            break
    
    # ✅ 날짜 가져오기
    article_date = "❌ 날짜 없음"
    for selector in date_selector:
        date_tag = soup.select_one(selector)
        if date_tag:
            article_date = date_tag.get_text(strip=True).split()[0]  # 'YYYY-MM-DD' 형태로 저장
            break

    return article_content, article_date

def crawl_naver_news():
    """📌 네이버 경제 뉴스 크롤링 (100개)"""
    try:
        response = requests.get(URL, headers=HEADERS)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"❌ 요청 오류 발생: {e}")
        return []
    
    soup = BeautifulSoup(response.text, "html.parser")
    news_list = soup.select("div.sa_text a")[:150]  # ✅ 최대 150개 크롤링 후 필터링

    news_data = []
    for news in news_list:
        title = news.get_text(strip=True)
        link = news["href"]

        # URL 정리
        if not link.startswith("http"):
            article_url = "https://news.naver.com" + link
        else:
            article_url = link

        # 필요 없는 뉴스 필터링
        if "더보기" in title or len(title) < 5:
            continue
        
        # ✅ 기사 본문 + 날짜 가져오기
        article_content, article_date = get_article_content_and_date(article_url)
        if "❌ 본문" in article_content or "❌ 날짜" in article_date:
            continue

        news_data.append((title, article_url, article_content, article_date))

        # ✅ 뉴스 100개가 채워지면 종료
        if len(news_data) >= 100:
            break

    return news_data

def save_news_to_csv(news_data, filename="C:/pthon_basic/webcrolingProject/news_crawler_project/data/naver_news.csv"):
    """📌 크롤링한 뉴스 데이터를 CSV로 저장"""
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(["제목", "링크", "본문", "날짜"])
        writer.writerows(news_data)

    print(f"✅ {filename} 파일이 업데이트되었습니다! (100개 뉴스 저장)")


# ✅ 실행
start_time = time.time()
news_data = crawl_naver_news()
save_news_to_csv(news_data)
end_time = time.time()

print(f"⏳ 실행 완료! 걸린 시간: {round(end_time - start_time, 2)}초")


✅ C:/pthon_basic/webcrolingProject/news_crawler_project/data/naver_news.csv 파일이 업데이트되었습니다! (100개 뉴스 저장)
⏳ 실행 완료! 걸린 시간: 8.95초
